# Vivino project
Author : Alphonse Doutriaux, march 2018

## 0. Imports and functions definitions

#### Import libs

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
import multiprocessing
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.metrics import r2_score
from joblib import Parallel, delayed
from xgboost import XGBRegressor

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### Import data, concateante wines and prices, split into train and test

In [2]:
df = pd.read_csv("./data/train.csv", index_col="vintage_id")

#### Custom functions

In [3]:
def premium_identifier(string):
    pgcc = re.search(r"[Pp]remier\s[Gg]rand\s[Cc]ru\s[Cc]lass[eé]", string) # matches the expression "Premier Grand Cru Classé" regardless of the case 
    gcc = re.search(r"[Gg]rand\s[Cc]ru\s[Cc]lass[eé]", string) # matches the expression "Grand Cru Classé" regardless of the case 
    gc = re.search(r"[Gg]rand\s[Cc]ru", string) # matches the "Grand Cru" expression regardless of the case
    bdb = re.search(r"[Bb]lanc\s[Dd]e\s[Bb]lanc", string) # matches the "Blanc de Blancs" expression regardless of the case
    pc = re.search(r"([Pp]remier|[1][a-z]*)\s[Cc]ru", string) # matches the "Premier Cru" expression regardless of the case

    if pgcc and gcc and gc:
        return("Premier Grand Cru Classé")
    elif not pgcc and gcc and gc:
        return("Grand Cru Classé")
    elif not pgcc and not gcc and gc:
        return("Grand Cru")
    elif bdb:
        return("Blanc de Blancs")
    elif pc:
        return("Premier Cru")
    else:
        return(np.nan)

In [4]:
def winery_category_identifier(string):
    domaine = re.search(r"([Dd]omaine|[Dd]omenico)", string) # matches the expression "Domaine" 
    chateau = re.search(r"([Cc]h[aâ]teau|[Cc]astle|Castello)", string) # matches the expression "Chateau" regardless of the case 
    
    if domaine :
        return("Domaine")
    elif chateau:
        return("Château")
    else:
        return(np.nan)

In [5]:
def region_cleaner(string):
    
    # this the the list of french AOCs
    region_list = ['alsace chasselas ou gutedel', 'alsace gewurztraminer', 'alsace grand cru', 'alsace muscat', 'alsace pinot noir', 'alsace pinot ou klevner', 'alsace riesling', 'alsace sylvaner', 'alsace tokay-pinot gris', "cremant d'alsace", 'beaujolais', 'beaujolais-villages', 'beaujolais superieur', 'brouilly', 'chenas', 'chiroubles', 'cote de brouilly', 'coteaux du lyonnais', 'fleurie', 'julienas', 'morgon', 'moulin a vent', 'regnie', 'saint-amour', 'barsac', 'blaye', 'bordeaux', 'bordeaux clairet', 'bordeaux cotes de francs', 'bordeaux rose', 'bordeaux sec', 'bordeaux superieur', 'cadillac', 'canon-fronsac', 'cerons', 'cremant de bordeaux', 'cotes de blaye', 'cotes de bourg', 'cotes de castillon', 'entre-deux-mers', 'fronsac', 'graves', 'graves de vayres', 'graves superieures', 'haut-medoc', 'lalande de pomerol', 'listrac-medoc', 'loupiac', 'margaux', 'medoc', 'montagne saint-emilion', 'moulis en medoc', 'pauillac', 'pessac leognan', 'pomerol', 'premieres cotes de bordeaux', 'premieres cotes de blaye', 'puisseguin saint-emilion', 'saint-emilion', 'saint-emilion grand cru', 'saint-estephe', 'saint-georges saint-emilion', 'saint-julien', 'sainte-croix-du-mont', 'sainte-foix-bordeaux', 'sauternes', 'aloxe-corton', 'auxey-duresses', 'batard-montrachet', 'beaune', 'bienvenue batard-montrachet ', 'blagny', 'bonnes-mares', 'bourgogne', 'bourgogne aligote', 'bourgogne aligote bouzeron', 'bourgogne chitry', 'bourgogne claret', 'bourgogne cote chalonnaise', 'bourgogne cote saint-jacques', "bourgogne cotes d'auxerre", 'bourgogne cotes du couchois', 'bourgogne coulanges-la-vineuse', 'bourgogne epineuil ', 'bourgogne grand ordinaire ', 'bourgogne hautes-cotes de nuits', 'bourgogne hautes-cotes de beaune', 'bourgogne la chapelle notre-dame', 'bourgogne le chapitre', 'bourgogne montrecul', 'bourgogne mousseux', 'bourgogne passetoutgrain', 'bourgogne vezelay', 'chablis', 'chablis grand cru', 'chablis premier cru', 'chambertin', 'chambertin-clos de beze', 'chapelle-chambertin', 'charmes-chambertin', 'chambolle musigny', 'chassagne-montrachet', 'chevalier-montrachet', 'chorey-les-beaune', 'clos de la roche', 'clos de tart', 'clos de vougeot', 'clos des lambrays', 'clos saint-denis', 'corton', 'corton-charlemagne', 'cote de beaune', 'cote de beaune villages', 'cote de nuits-villages', 'cremant de bourgogne', 'criots batard-montrachet ', 'echezeaux', 'fixin', 'gevrey-chambertin', 'givry', 'grands-echezeaux', 'griotte-chambertin', 'irancy', 'ladoix', 'la grande rue', 'la tache', 'la romanee', 'latricieres-chambertin', 'macon', 'macon superieur', 'macon-villages', 'maranges', 'marsannay', 'mazis-chambertin', 'mazoyeres-chambertin', 'mercurey', 'meursault', 'montagny', 'monthelie', 'montrachet', 'morey-saint-denis', 'nuits-saint-georges', 'pernand-vergelesses', 'petit chablis', 'pommard', 'pouilly-fuisse', 'pouilly-loche', 'pouilly-vinzelles', 'puligny-montrachet', 'richebourg', 'romanee-conti', 'romanee-saint-vivant', 'ruchottes-chambertin', 'rully', 'saint-aubin', 'saint-bris', 'saint-romain', 'saint-veran', 'santenay', 'savigny-les-beaune', 'vire-clesse', 'volnay', 'vosne-romanee', 'vougeot', 'beaumes de venise', 'chateau-grillet', 'chateauneuf-du-pape', 'clairette de die', 'condrieu', 'cornas', 'cote rotie', 'coteaux de pierrevert', 'coteaux du tricastin', 'cotes du luberon', 'cotes du rhone', 'cotes du rhone-villages', 'cotes du ventoux', 'cotes du vivarais aovdqs', 'crozes-hermitage', 'gigondas', 'hermitage', 'lirac', 'muscat de beaumes-de-venise', 'saint-joseph', 'saint peray', 'tavel', 'vacqueyras', 'vinsobres', 'banyuls', 'banyuls grand cru', 'blanquette de limoux', 'cabardes aovdqs', 'clairette de bellegarde', 'clairette du languedoc', 'collioure', 'corbieres', 'costieres de nimes', 'coteaux du languedoc', 'cotes de la malepere', 'cotes du roussillon', 'cotes du roussillon-villages', 'faugeres', 'fitou', 'limoux', 'maury', 'minervois', 'muscat de lunel', 'muscat de mireval', 'muscat de rivesaltes', 'muscat de saint-jean de minervois', 'picpoul de pinet', 'rivesaltes', 'saint-chinian', 'anjou', 'anjou-coteaux de la loire', 'anjou-gamay', 'anjou-villages', 'bonnezeaux', 'bourgueil', "cabernet d'anjou", 'chateaumeillant aovdqs', 'chaume', 'cheverny', 'chinon', 'cote roannaise', "coteaux d'ancenis", "coteaux de l'aubance", 'coteaux du giennois aovdqs', 'coteaux du layon', 'coteaux du loir', 'coteaux du vendomois aovdqs', "cotes d'auvergne aovdqs", 'cotes du forez aovdqs', 'cour-cheverny', 'fiefs vendeens aovdqs', 'gros plant aovdqs', 'haut-poitou aovdqs', 'jasnieres', 'menetou-salon', 'montlouis', 'muscadet', 'muscadet cotes de grand-lieu', 'muscadet de sevre-et-maine', 'muscadet des coteaux de la loire', 'quarts de chaume', 'pouilly-fume', 'pouilly-sur-loire', 'quincy', 'reuilly', 'rose de loire', "rose d'anjou", 'saint-nicolas-de-bourgueil', 'saint-pourcain aovdqs', 'sancerre', 'saumur', 'saumur sec blanc', 'saumur-champigny', 'savennieres', 'savennieres coulee-de-serrant', 'savennieres roches-aux-moines', 'touraine', 'touraine amboise', 'touraine-azay-le-rideau', 'touraine-mesland', 'valencay aovdqs', 'vouvray', 'bandol', 'bellet', 'cassis', "coteaux d'aix", 'coteaux varois en provence', 'cotes de provence', 'palette', 'ajaccio', 'muscat du cap corse', 'patrimonio', 'vin de corse', 'bearn', 'bergerac', 'bergerac sec', 'buzet', 'cahors', 'cotes de bergerac', 'cotes de bergerac moelleux', 'cotes de duras', 'cotes de saint-mont aovdqs', 'cotes du marmandais', 'fronton', 'gaillac', 'haut-montravel', 'irouleguy', 'jurancon', 'jurancon sec', 'madiran', 'marcillac', 'monbazillac', 'montravel', 'pacherenc du vic-bilh', 'pecharmant', 'saussignac', 'tursan aovdqs', 'arbois', 'arbois pupillin', 'bugey aovdqs', 'chateau-chalon', 'champagne', 'coteaux champenois', 'rose des riceys', 'cotes de toul aovdqs', 'moselle aovdqs', 'cotes du jura', 'cremant du jura', "l'etoile", 'pineau des charentes', 'roussette de savoie', 'vin de savoie']
    match_list = []
    
    cleaned_string = str(unicodedata.normalize('NFKD', string).encode('ASCII', 'ignore'))
    
    for region in region_list:
        if re.search(region, cleaned_string, re.IGNORECASE):
            # we create a list in which every element is the length of the match between the input string and each region
            match_length = re.search(region, cleaned_string, re.IGNORECASE).span()[1] - re.search(region, cleaned_string, re.IGNORECASE).span()[0]
            match_list.append(match_length)
        else:
            match_list.append(0)
            
    if max(match_list) == 0: # if no region has matched
        return(string.title())
    else: # if a region has matched, the region whose match had the maximum length is the best fit
        return(region_list[match_list.index(max(match_list))].title())

In [6]:
# to be used for in-house cross-validation
def fit_model(traini, model):
    model.fit(X_train.iloc[traini], y_train.iloc[traini])
    return(model)

## 1. Preprocessing

#### Rough data cleansing
* Drop `vintage_name` column, which is equal to `wine_name` + `wine_year` (except for champagne and affiliates)

In [7]:
df = df.drop(['vintage_name'], axis=1)

* Replace `U.V.` and `N.V.` in `wine_year` with `2018` (this problem mainly concerns champagne and sparkling wines)

In [8]:
df['wine_year'] = df['wine_year'].replace("U.V.", 2018)
df['wine_year'] = df['wine_year'].replace("N.V.", 2018)
df['wine_year'] = df['wine_year'].astype(int)

#### Identify premium wines using the custom *premium_identifier* function

In [9]:
for index in df.index:
    premium_category = premium_identifier(df.loc[index, 'wine_name'])
    df.loc[index, 'premium_category'] = premium_category
df['premium_category'] = df['premium_category'].replace(np.nan, "None")

#### Identify winery category using the custom *winery_category_identifier* function

In [10]:
for index in df.index:
    winery_category = winery_category_identifier(df.loc[index, 'winery_name'])
    df.loc[index, 'winery_category'] = winery_category
df['winery_category'] = df['winery_category'].replace(np.nan, "None")

#### Reduce the number of region names with *soft grouping* and delete regions apearing only once

In [11]:
# this function allows to reduce the number of wine_region_names by 25%
for index in df.index:
    df.loc[index, "wine_region_name"] = region_cleaner(df.loc[index, "wine_region_name"])

In [12]:
# delete rows whose region appears only once in the entire dataset
value_counts = df['wine_region_name'].value_counts()
regions_to_remove = value_counts[value_counts == 1].index.tolist()

for index in df.index: 
    if df.loc[index, 'wine_region_name'] in regions_to_remove: # for each wine, check if its region belongs to the to-be-removed regions
        df = df.drop(index, axis=0) # if yes, drop this wine

#### Replace `NaN` values in `wine_region_production_volume` and `wine_region_surface` with median values

In [13]:
df['wine_region_production_vol'] = df['wine_region_production_vol'].fillna(df['wine_region_production_vol'].mean())
df['wine_region_surface'] = df['wine_region_surface'].fillna(df['wine_region_surface'].mean())

#### Remove outliers in `ratings_list` column

In [14]:
# to be executed only once
df = df[df['vintage_ratings_count'] < df['vintage_ratings_count'].mean() + 3 * df['vintage_ratings_count'].std()]
df = df[df['vintage_ratings_count'] > df['vintage_ratings_count'].mean() - 3 * df['vintage_ratings_count'].std()]

#### Add an `is_brut` column for Champagne

In [15]:
for index in df.index:
    if re.search(r"Brut", df.loc[index, 'wine_name']):
        df.loc[index, "is_brut"] = 1
    else:
        df.loc[index, "is_brut"] = 0
# average Brut price 70, average champagne price 77

#### Prepare for the regression

In [16]:
data = df.copy()

In [17]:
df = df.drop(['wine_name', 'winery_name', 'wine_id'], axis=1) # if one should be kept, probably winery_name

In [18]:
cols_to_encode = ['wine_region_name', 'premium_category', 'winery_category']
for col in cols_to_encode:
    df[col] = df[col].astype('category', categories = df[col].unique().tolist())

df = pd.get_dummies(df, columns = cols_to_encode)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
X_train = df.drop(['price'], axis=1)
y_train = df['price']

## 2. XGBoost and RandomForest regressions
### 2.1. XGBoost
* GridsearchCV and hyperparameter tuning

In [36]:
# this script takes circa. 20s

n_estimators = [20]
learning_rate = [0.1, 0.3, 0.5]
gamma = [0]
max_depth = [5, 7, 9]
colsample_bytree = np.arange(0.2, 1, 0.2)

param_grid_xgb = dict(n_estimators=n_estimators, learning_rate=learning_rate, gamma=gamma, max_depth=max_depth, colsample_bytree=colsample_bytree)

xgb_gs = GridSearchCV(XGBRegressor(), param_grid=param_grid_xgb, n_jobs=-1)
grid_result_xgb = xgb_gs.fit(X_train,y_train)

print("Best R2 score using XGBoost: {:.3}".format(xgb_gs.best_score_))
print(xgb_gs.best_params_)

Best R2 score using XGBoost: 0.684
{'colsample_bytree': 0.4, 'gamma': 0, 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 20}


* Model fitting and in-house cross-validation

In [37]:
xgb = XGBRegressor(n_estimators=20,
                  learning_rate=xgb_gs.best_params_['learning_rate'],
                  gamma=xgb_gs.best_params_['gamma'],
                  max_depth=xgb_gs.best_params_['max_depth'],
                  colsample_bytree=xgb_gs.best_params_['colsample_bytree']
                  )

In [38]:
num_cores = multiprocessing.cpu_count()
kf = list(KFold(len(X_train),num_cores, shuffle=False, random_state=None))
xgbs = Parallel(n_jobs=num_cores)(delayed(fit_model)(traini, xgb) for traini,_ in kf)
preds = np.empty_like(y_train)

for i,(_,testi) in enumerate(kf):
    preds[testi] = xgbs[i].predict(X_train.iloc[testi])
    
print("R2 score using XGBoost on train set: {:.3}".format(r2_score(y_train, preds)))

R2 score using XGBoost on train set: 0.681


### 2.2 RandomForest

* GridSearch to tune hyperparameters

In [23]:
# this script takes circa. 20s

n_estimators = [20]
max_features = np.arange(0.1, 1, 0.1)
max_depth = [2, 3, 4, 5, 6]

param_grid_rf = dict(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth)

rf_gs = GridSearchCV(RandomForestRegressor(), param_grid=param_grid_rf, n_jobs=-1)
grid_result_rf = rf_gs.fit(X_train,y_train)

print("Best R2 score using RandomForest: {:.3}".format(rf_gs.best_score_))
print(rf_gs.best_params_)

Best R2 score using RandomForest: 0.643
{'max_depth': 6, 'max_features': 0.6, 'n_estimators': 20}


* Model fitting and in-house cross-validation

In [24]:
rf = RandomForestRegressor(n_estimators=1000,
                          max_features=rf_gs.best_params_['max_features'],
                          max_depth=rf_gs.best_params_['max_depth'])

In [25]:
num_cores = multiprocessing.cpu_count()
kf = list(KFold(len(X_train),num_cores, shuffle=False, random_state=None))
rfrs = Parallel(n_jobs=num_cores)(delayed(fit_model)(traini, rf) for traini,_ in kf)
preds = np.empty_like(y_train)

for i,(_,testi) in enumerate(kf):
    preds[testi] = rfrs[i].predict(X_train.iloc[testi])
    
print("R2 score using RandomForest on train set: {:.3}".format(r2_score(y_train, preds)))

R2 score using RandomForest on train set: 0.644


### 2.3. Blending

In [26]:
X_train_bl, X_test_bl, y_train_bl, y_test_bl = train_test_split(X_train, y_train, test_size=0.2)

xgb.fit(X_train_bl, y_train_bl)
rf.fit(X_train_bl, y_train_bl)

xgb_preds = xgb.predict(X_test_bl)
rf_preds = rf.predict(X_test_bl)

preds_table = pd.DataFrame({"XGBoost":xgb_preds, "RandomForest":rf_preds}, index=X_test_bl.index)

# We use a LinearRegression to determine how to blend the models predictions

lr_blending = LinearRegression(n_jobs=-1)
lr_blending.fit(preds_table, y_test_bl)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

## 3. Scoring on test set

In [27]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features=0.6, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [28]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=20,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

#### Import data

In [29]:
test = pd.read_csv("./data/test.csv", index_col="vintage_id")

#### Preprocessing

In [30]:
test = test.drop(['vintage_name'], axis=1)

# replace NaN original values with 2018 (because it ùainly concerns champagne)
test['wine_year'] = test['wine_year'].replace("U.V.", 2018)
test['wine_year'] = test['wine_year'].replace("N.V.", 2018)
test['wine_year'] = test['wine_year'].astype(int)

# round ratings average
for index in test.index:
    test.loc[index, 'vintage_ratings_average'] = round(test.loc[index, 'vintage_ratings_average'],1)

# identify premium wines
for index in test.index:
    premium_category = premium_identifier(test.loc[index, 'wine_name'])
    test.loc[index, 'premium_category'] = premium_category
test['premium_category'] = test['premium_category'].replace(np.nan, "None")

# identify winery category
for index in test.index:
    winery_category = winery_category_identifier(test.loc[index, 'winery_name'])
    test.loc[index, 'winery_category'] = winery_category
test['winery_category'] = test['winery_category'].replace(np.nan, "None")

# this function allows to reduce the number of wine_region_names by 25%
for index in test.index:
    test.loc[index, "wine_region_name"] = region_cleaner(test.loc[index, "wine_region_name"])

# delete rows whose region appears only once in the entire dataset
value_counts = test['wine_region_name'].value_counts()
regions_to_remove = value_counts[value_counts == 1].index.tolist()
for index in test.index: 
    if test.loc[index, 'wine_region_name'] in regions_to_remove:
        test = test.drop(index, axis=0)

# replace NaN values with mean in region features
test['wine_region_production_vol'] = test['wine_region_production_vol'].fillna(df['wine_region_production_vol'].mean())
test['wine_region_surface'] = test['wine_region_surface'].fillna(df['wine_region_surface'].mean())

# remove extreme values based on original dataframe value repartition (to be executed once only)
test = test[test['vintage_ratings_count'] < df['vintage_ratings_count'].mean() + 3 * df['vintage_ratings_count'].std()]
test = test[test['vintage_ratings_count'] > df['vintage_ratings_count'].mean() - 3 * df['vintage_ratings_count'].std()]

# add is_brut column
for index in test.index:
    if re.search(r"Brut", test.loc[index, 'wine_name']):
        test.loc[index, "is_brut"] = 1
    else:
        test.loc[index, "is_brut"] = 0

test = test.drop(['wine_name', 'winery_name', 'wine_id'], axis=1) # if one should be kept, probably winery_name

# one hot encoding
for col in cols_to_encode:
    test[col] = test[col].astype('category', categories = data[col].unique().tolist())
test = pd.get_dummies(test, columns = cols_to_encode)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


#### Predictions

In [31]:
X_test = test.drop(['price'], axis=1)
y_test = test['price']

In [32]:
xgb_preds_test = xgb.predict(X_test)
rf_preds_test = rf.predict(X_test)

preds_table_final = pd.DataFrame({"XGBoost":xgb_preds_test, "RandomForests":rf_preds_test}, index=test.index)
preds = lr_blending.predict(preds_table_final)

In [33]:
print("Final R2 score (after blending): {:.3}".format(r2_score(y_test, preds)))

Final R2 score (after blending): 0.656
